In [2]:
import pandas as pd
import numpy as np
import sklearn
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import importlib
import warnings

warnings.filterwarnings("ignore");

print('Pandas version:', pd.__version__);
print('Numpy version:', np.__version__);
print('MatplotLib version:', mpl.__version__);
print('Sklearn version:', sklearn.__version__);
print('Seaborn version:', sns.__version__);

Pandas version: 1.3.3
Numpy version: 1.20.1
MatplotLib version: 3.5.3
Sklearn version: 0.24.2
Seaborn version: 0.12.1


# Reading the datasets for train and test

The dataset should contain all columns used as predictors the terminal states of the compounds as integers in increasing order (column name here: "Terminal State(int)").

### The following lists need to be specified:

    global_features: list of all column names used as predictors

    features_to_be_normed: list of all column names that should be scaled

In [ ]:
df = pd.read_csv(data_path);

Define train, val and test sets

In [ ]:
df_train = TO_BE_SET;
df_val = TO_BE_SET;

df_test = TO_BE_SET;
df_train_final = TO_BE_SET;

# Scale the data

In [ ]:
from sklearn.preprocessing import StandardScaler

print("Scaling the initial train and val ...");

sc = StandardScaler()
train_scaler = sc.fit(df_train[features_to_be_normed]);
df_train[features_to_be_normed] = train_scaler.transform(df_train[features_to_be_normed]);
df_val[features_to_be_normed] = train_scaler.transform(df_val[features_to_be_normed]);

print("Scaling the train and test ...");
sc = StandardScaler()
final_train_scaler = sc.fit(df_train_final[features_to_be_normed]);
import pickle
scalerfile = 'scaler_onlyassays.sav'
pickle.dump(final_train_scaler, open(scalerfile, 'wb'))
final_train_scaler = pickle.load(open(scalerfile, 'rb'))

df_train_final[features_to_be_normed] = final_train_scaler.transform(df_train_final[features_to_be_normed]);
df_test[features_to_be_normed] = final_train_scaler.transform(df_test[features_to_be_normed]);

# Now the MLP

In [ ]:
import torch
from torch.utils.data import DataLoader
from sklearn import metrics
 
if torch.cuda.is_available():
    print('use GPU');
    device='cuda';
else:
    print('use CPU');
    device='cpu';
    
from coral_pytorch.losses import corn_loss

mod = importlib.import_module('Predictor_onlyGlobalFeats_wdropout');
importlib.reload(mod);
Predictor_onlyGlobalFeats_wdropout = getattr(mod, 'Predictor_onlyGlobalFeats_wdropout');

In [ ]:
def get_data( global_features, labels):

    bPKs= [];
    global_feat_list = [];

    for i, smi in enumerate(labels):
        
        bPKs.append(int(labels[i]));
        global_feat_list.append(torch.from_numpy(global_features[i,:])); 
        
    return  global_feat_list, bPKs;

In [ ]:
def collate(sample):
    global_feats, labels = map(list,zip(*sample));

    global_feats = torch.stack([torch.tensor(tmp) for tmp in global_feats]);

    return  global_feats, torch.tensor(labels);

In [ ]:
def calc_precentile_borders(data, num_ints):
    borders = [];
    print("Interval borders:");
    percs = 100/num_ints;
    
    for tmp_int in range(num_ints-1):
        tmp_perc = np.percentile(data, (tmp_int+1)*percs);
        borders.append(tmp_perc);
        print(tmp_perc);
    
    return borders;

In [ ]:
def fit_gat(params):    
   
    print(params);
    num_tasks = 5;
    model = Predictor_onlyGlobalFeats_wdropout( global_feats=len(global_features), predictor_dropout=params["dropout"], num_layers=params["depth"],
                                       n_tasks=num_tasks, predictor_hidden_feats=params["predictor_hidden_feats"]);
    
    model = model.to(device);

    global_feat_train, bPKs_train = get_data( df_train[global_features].to_numpy(dtype=np.float32), 
                                                                 df_train["Terminal State(int)"].to_list());
    global_feat_val, bPKs_val = get_data( df_val[global_features].to_numpy(dtype=np.float32),
                                                           df_val["Terminal State(int)"].to_list());

    train_data = list(zip( global_feat_train, bPKs_train)).copy();
    train_loader = DataLoader(train_data, batch_size=int(params["batch_size"]), shuffle=True, collate_fn=collate, pin_memory=True, num_workers=1);
    
    val_data = list(zip( global_feat_val, bPKs_val)).copy();
    val_loader = DataLoader(val_data, batch_size=256, shuffle=False, collate_fn=collate, drop_last=False);

    optimizer = torch.optim.AdamW(model.parameters(), weight_decay=params["weight_decay"]);
    scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer, base_lr=0.0001, max_lr=0.01,mode="triangular2",cycle_momentum=False, step_size_up=2*len(train_loader));

    #######################
    ### Train the model ###
    #######################
    epoch_losses = [];
    epoch_aucs = [];
    
    #early stopping params
    num_no_improvements = 0;
    best_val = 0;
    patience = 10;
    
    for epoch in range(1,100):
        model.train();
        epoch_loss = 0;
        epoch_acc = 0;
        for i, (global_feats, labels) in enumerate(train_loader):

            labels = labels.to(device);
            global_feats = global_feats.to(device);

            logits, _ = model( global_feats);
            loss = corn_loss(logits, labels.float(), num_classes=num_tasks);
            
            optimizer.zero_grad();
            loss.backward();
            optimizer.step();
            epoch_loss += loss.detach().item();
            scheduler.step();
            
        epoch_losses.append(epoch_loss);

        #######################
        ### Valid the model ###
        #######################
        model.eval();
        probs = [];
        for i, ( global_feats, labels) in enumerate(val_loader):
            global_feats = global_feats.to(device);
            labels = labels.to(device);
            logits, prob = model( global_feats);
            tmp_probs = prob.cpu().detach().numpy();    
            probs = probs + list(tmp_probs[:,1]);
        
        fpr, tpr, _ = metrics.roc_curve(df_val["bPK"].to_list(), probs);
        tmp_val = metrics.auc(fpr, tpr);
        epoch_aucs.append(tmp_val);
        
        print(optimizer.param_groups[0]['lr']);

        
        if tmp_val > best_val:
            num_no_improvements = 0;
            best_val = tmp_val;
        else:
            num_no_improvements = num_no_improvements + 1;
            if num_no_improvements>patience:
                print("Early stopping here ...");
                break;
        
        epoch_aucs.append(tmp_val);
        print(tmp_val);
        
    print("Best val: " + repr(best_val));
    return 1-best_val;

#-------------------------------------------------------------------------------
def fit_gat_final(params):
    import copy
    
    num_tasks = 5;
    print(params);
    model = Predictor_onlyGlobalFeats_wdropout( global_feats=len(global_features), predictor_dropout=params["dropout"], num_layers=params["depth"],
                                       n_tasks=num_tasks, predictor_hidden_feats=params["predictor_hidden_feats"]);

    model = model.to(device);

    global_feat_train, bPKs_train = get_data( df_train_final[global_features].to_numpy(dtype=np.float32), 
                                                                 df_train_final["Terminal State(int)"].to_list());
    
    global_feat_val, bPKs_val = get_data( df_test[global_features].to_numpy(dtype=np.float32),
                                                           df_test["Terminal State(int)"].to_list());
    print("Data loaded ...");
    
    train_data = list(zip( global_feat_train, bPKs_train)).copy();
    train_loader = DataLoader(train_data, batch_size=int(params["batch_size"]), shuffle=True, collate_fn=collate, pin_memory=True, num_workers=1);
    
    val_data = list(zip( global_feat_val, bPKs_val)).copy();
    val_loader = DataLoader(val_data, batch_size=256, shuffle=False, collate_fn=collate, drop_last=False);

    optimizer = torch.optim.AdamW(model.parameters(), weight_decay=params["weight_decay"]);
    scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer, base_lr=0.0001, max_lr=0.01, mode="triangular2", step_size_up=2*len(train_loader),cycle_momentum=False);
    model.train();

    epoch_losses = [];
    epoch_accuracies = [];
    
    #early stopping params
    num_no_improvements = 0;
    best_val = 0;
    patience = 10;
    
    
    for epoch in range(1,100):
        model.train();
        epoch_loss = 0;
        for i, (global_feats, labels) in enumerate(train_loader):

            labels = labels.to(device);
            global_feats = global_feats.to(device);
            

            logits, probas = model( global_feats);
            loss = corn_loss(logits, labels.float(), num_classes=num_tasks);
            
            optimizer.zero_grad();
            loss.backward();
            optimizer.step();
            epoch_loss += loss.detach().item();
            scheduler.step();
      
        epoch_loss /= (i + 1)
        if epoch % 1 == 0:
            print(f"epoch: {epoch}, LOSS: {epoch_loss:.3f}");
        epoch_losses.append(epoch_loss);
        
        #######################
        ### Valid the model ###
        #######################
        model.eval();
        probs = [];
        for i, ( global_feats, labels) in enumerate(val_loader):            
            labels = labels.to(device);
            global_feats = global_feats.to(device);
            _, prob = model(global_feats);
            tmp_probs = prob.cpu().detach().numpy();
            probs = probs + list(tmp_probs[:,1]);
            
        fpr, tpr, _ = metrics.roc_curve(df_test["bPK"].to_list(), probs);
        tmp_val = metrics.auc(fpr, tpr);
        print("Learning rate: " + str(optimizer.param_groups[0]['lr']));

        if tmp_val > best_val:
            num_no_improvements = 0;
            best_val = tmp_val;
            best_model = copy.deepcopy(model);
        else:
            num_no_improvements = num_no_improvements + 1;
            if num_no_improvements>patience:
                print("Early stopping here ...");
                break;
        
        epoch_accuracies.append(tmp_val);
        print(tmp_val);
      
    print("Best val: " + repr(best_val));
    return best_model, best_val;

### Run the training

In [ ]:
import copy
final_params = {'batch_size': 32, 'depth': 7, 'dropout': 0.3, 'predictor_hidden_feats': 256, 'weight_decay': 0.01};

auc_threshold = 0.5;
best_auc = 0;
i = 0;
while i < 10:
    model, auc = fit_gat_final(final_params);
    if auc>auc_threshold:
        torch.save(model.state_dict(), "weights_" + str(i) + ".pth");
        i = i + 1;
    if auc > best_auc:
        best_auc = auc;
        best_model = copy.deepcopy(model);

# Testing 

In [ ]:
import predict_onlyassays
mod = importlib.reload(predict_onlyassays);

df_test = df.iloc[int(0.75*df.shape[0]): ,:];
preds = mod.predict(df_test.copy(), ".");

In [ ]:
from sklearn import metrics

fpr, tpr, thresholds = metrics.roc_curve(df_test["bPK"], preds);
plt.plot(fpr, tpr, color="black", linewidth=2);
plt.plot(np.arange(0, 1, 0.01), np.arange(0,1, 0.01), color="red", linewidth=0.5);
plt.xlabel("False positive rate", fontsize=12);
plt.ylabel("True positive rate", fontsize=12);
print("AUC");
print(metrics.auc(fpr, tpr));

In [ ]:
precision, recall, thresholds = metrics.precision_recall_curve(df_test["bPK"], preds);

f, axs = plt.subplots(1,3, figsize=(17,5));

axs[0].plot(recall[1:-1], precision[1:-1], color="black", linewidth=2);
axs[0].set_xlabel("Recall");
axs[0].set_ylabel("Precision");
#axs[0].set_yscale("log");

axs[1].plot(thresholds, precision[:-1], color="black", linewidth=2);
axs[1].set_xlabel("Threshold");
axs[1].set_ylabel("Precision");

axs[2].plot(thresholds, recall[:-1], color="black", linewidth=2);
axs[2].set_xlabel("Threshold");
axs[2].set_ylabel("Recall");

In [ ]:
from sklearn.calibration import calibration_curve

normed_preds = preds - np.min(preds);
normed_preds = normed_preds/np.max(normed_preds);

bPKs = df_test["bPK"].to_numpy();
prob_true, prob_pred = calibration_curve(bPKs, preds, n_bins=5, strategy="quantile");

y_tests_np = np.array(bPKs);
baseline_prob = len(y_tests_np[y_tests_np==1.0])/len(y_tests_np);
print("Baseline probability:" + repr(baseline_prob));

borders = calc_precentile_borders(preds, 5);

sns.regplot(x=prob_pred, y=prob_true, ci=0, order=0, color="black", lowess=True);
plt.xlabel("bPK score");
plt.ylabel("Probabilit of transition beyond PK");